
# Public Key Encryption
The two main method used for public key methods are RSA and ECC (Elliptic Curve Cryptography). With encryption Bob can use Alice's public key to encrypt data, and then she will use her private key to decrypt it:

<img src='graphics/g_public_04.png' width="800px">


## RSA
Overall, Bob generates two random prime numbers (p and q), and create a public modulus of:

N=p.q

Next Bob computes φ:

φ=(p−1).(q−1)

Bob then picks a public exponent of e and which does not share a factor with φ, and computes the private exponent as:

d=e^{−1}(modφ)

Bob will then use a public exponent (e) to cipher a message (M) with:

C=M^e (mod N)

To decrypt we use the private exponent (d) to decipher the ciphertext:

M=C^d (mod N)

Bob's public key is [e,N] and his private key is [d,N].

<img src='graphics/g_public_12.png' width="800px">


In [1]:
# https://asecuritysite.com/rsa/rsa12
from Crypto.Util.number import *
from Crypto import Random
import Crypto
import libnum

bits=60
msg="Hello"

p = Crypto.Util.number.getPrime(bits, randfunc=Crypto.Random.get_random_bytes)
q = Crypto.Util.number.getPrime(bits, randfunc=Crypto.Random.get_random_bytes)

n = p*q
PHI=(p-1)*(q-1)

e=65537
d=libnum.invmod(e,PHI)

m=  bytes_to_long(msg.encode('utf-8'))

c=pow(m,e, n)
res=pow(c,d ,n)

print ("Message=%s\np=%s\nq=%s\n\nd=%d\ne=%d\nN=%s\n\nPrivate key (d,n)\nPublic key (e,n)\n\ncipher=%s\ndecipher=%s" % (msg,p,q,d,e,n,c,(long_to_bytes(res))))

Message=Hello
p=1015811308611355393
q=755523213775410319

d=589399628769593821059839430475148801
e=65537
N=767469024471456365658376241308500367

Private key (d,n)
Public key (e,n)

cipher=302667991570245321224673969207643936
decipher=b'Hello'



> Change the size of the prime numbers to 100 bits, and verify that the program works.

> Change the "d=libnum.invmod(e,PHI)" to "pow(e,-1,PHI)" and show that the code still works correctly.

A more complex example is:

In [ ]:
# https://asecuritysite.com/hazmat/hashnew8
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
import binascii
import sys


size=1024
message = b"Hello world"

if (len(sys.argv)>1):
	message=str(sys.argv[1]).encode()
if (len(sys.argv)>2):
	size=int(sys.argv[2])

print("Message: ",message)
print("Key size: ",size)
try:
  private_key = rsa.generate_private_key(public_exponent=65537,key_size=size)


  pub = private_key.public_key()

  ciphertext = pub.encrypt(message,padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))

  print ("\nCiphertext:\n",binascii.b2a_hex(ciphertext).decode())

  plaintext = private_key.decrypt(ciphertext,padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))
  print("\nDecrypted Message: ",plaintext.decode())

  print("\n=== Private Key PEM format ===")
  pem = private_key.private_bytes(encoding=serialization.Encoding.PEM,format=serialization.PrivateFormat.PKCS8,encryption_algorithm=serialization.NoEncryption())


  print ("\nPrivate key (PEM):\n",pem.decode())


  pem = pub.public_bytes(encoding=serialization.Encoding.PEM,format=serialization.PublicFormat.SubjectPublicKeyInfo).decode("utf-8")



  print("\n=== Public Key format ===")
  print ("Public key (PEM):\n",pem)


except Exception as e:
    print(e)

> Implement the code with  2K RSA key, and verify its operation.

> Implement the program using a DER format for the keys. When you view the keys, what do you observe from the first two hex characters of the hex value of the keys?

> The code uses SHA-256 for the padding method. Change the implementation of this to SHA-1, and verify the operation.

## ECIES


<img src='graphics/ecc3.png' width="800px">

In [2]:
# https://asecuritysite.com/ecc/ecc3
import collections
import hashlib
import random
import binascii
import sys
from Crypto.Cipher import AES
import Padding


def enc_long(n):
    '''Encodes arbitrarily large number n to a sequence of bytes.
    Big endian byte order is used.'''
    s = ""
    while n > 0:
        s = chr(n & 0xFF) + s
        n >>= 8
    return s




# Padding for the input string --not
# related to encryption itself.
BLOCK_SIZE = 16  # Bytes
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * \
                chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)
unpad = lambda s: s[:-ord(s[len(s) - 1:])]


def encrypt(plaintext,key, mode):
	encobj = AES.new(key,mode)
	return(encobj.encrypt(plaintext))

def decrypt(ciphertext,key, mode):
	encobj = AES.new(key,mode)
	return(encobj.decrypt(ciphertext))


EllipticCurve = collections.namedtuple('EllipticCurve', 'name p a b g n h')

curve = EllipticCurve(
    'secp256k1',
    # Field characteristic.
    p=0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f,
    # Curve coefficients.
    a=0,
    b=7,
    # Base point.
    g=(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8),
    # Subgroup order.
    n=0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141,
    # Subgroup cofactor.
    h=1,
)


# Modular arithmetic ##########################################################

def inverse_mod(k, p):
    """Returns the inverse of k modulo p.
    This function returns the only integer x such that (x * k) % p == 1.
    k must be non-zero and p must be a prime.
    """
    if k == 0:
        raise ZeroDivisionError('division by zero')

    if k < 0:
        # k ** -1 = p - (-k) ** -1  (mod p)
        return p - inverse_mod(-k, p)

    # Extended Euclidean algorithm.
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = p, k

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    gcd, x, y = old_r, old_s, old_t

    assert gcd == 1
    assert (k * x) % p == 1

    return x % p


# Functions that work on curve points #########################################

def is_on_curve(point):
    """Returns True if the given point lies on the elliptic curve."""
    if point is None:
        # None represents the point at infinity.
        return True

    x, y = point

    return (y * y - x * x * x - curve.a * x - curve.b) % curve.p == 0


def point_neg(point):
    """Returns -point."""
    assert is_on_curve(point)

    if point is None:
        # -0 = 0
        return None

    x, y = point
    result = (x, -y % curve.p)

    assert is_on_curve(result)

    return result


def point_add(point1, point2):
    """Returns the result of point1 + point2 according to the group law."""
    assert is_on_curve(point1)
    assert is_on_curve(point2)

    if point1 is None:
        # 0 + point2 = point2
        return point2
    if point2 is None:
        # point1 + 0 = point1
        return point1

    x1, y1 = point1
    x2, y2 = point2

    if x1 == x2 and y1 != y2:
        # point1 + (-point1) = 0
        return None

    if x1 == x2:
        # This is the case point1 == point2.
        m = (3 * x1 * x1 + curve.a) * inverse_mod(2 * y1, curve.p)
    else:
        # This is the case point1 != point2.
        m = (y1 - y2) * inverse_mod(x1 - x2, curve.p)

    x3 = m * m - x1 - x2
    y3 = y1 + m * (x3 - x1)
    result = (x3 % curve.p,
              -y3 % curve.p)

    assert is_on_curve(result)

    return result


def scalar_mult(k, point):
    """Returns k * point computed using the double and point_add algorithm."""
    assert is_on_curve(point)

    if k % curve.n == 0 or point is None:
        return None

    if k < 0:
        # k * point = -k * (-point)
        return scalar_mult(-k, point_neg(point))

    result = None
    addend = point

    while k:
        if k & 1:
            # Add.
            result = point_add(result, addend)

        # Double.
        addend = point_add(addend, addend)

        k >>= 1

    assert is_on_curve(result)

    return result


# Keypair generation and ECDSA ################################################

def make_keypair():
    """Generates a random private-public key pair."""
    private_key = random.randrange(1, curve.n)
    public_key = scalar_mult(private_key, curve.g)

    return private_key, public_key



message="Hello"


dA, Qa = make_keypair()
print("Private key:", hex(dA))
print(("Public key: (0x{:x}, 0x{:x})".format(*Qa)))


print("\n\n=========================")

r = random.randint(0, 2**128)

rG = scalar_mult(r,curve.g)
S = scalar_mult(r,Qa)

print("Random value: " , r)
print("rG: " , rG)

print("\n\n======Symmetric key========")

print("Encryption key:",S[0],str(S[0]))
password='hello'

key = hashlib.sha256(str(S[0]).encode()).digest()

message = Padding.appendPadding(message,blocksize=Padding.AES_blocksize,mode=0)

ciphertext = encrypt(message.encode(),key,AES.MODE_ECB)




print("Encrypted:\t",binascii.hexlify(ciphertext))


Snew = scalar_mult(dA,rG)
key = hashlib.sha256(str(Snew[0]).encode()).digest()

text = decrypt(ciphertext,key,AES.MODE_ECB)


print("Decrypted:\t",Padding.removePadding(text.decode(),mode=0))

Private key: 0x8429f9e6136fcfa883130853defd319d931e25207749de92b5e12a80c8e62619
Public key: (0xa4581b11e2aa0249274374216d8fb965517e45e6bcc337f3fc679c2bfd84dce7, 0x4716b0eaf5f748af3921aac8455e623b96b590453f313b4d200419a801c11267)


Random value:  23353093005829895849666617279986636640
rG:  (38430190155405327152400342621384391216588051326127513706497722898874838651319, 32551409443475257236910193737311399537798158570670164100113247023590451377060)


======Symmetric key========
Encryption key: 34425016396310312658614745921144437662843062008454472115656949568073393878924 34425016396310312658614745921144437662843062008454472115656949568073393878924
Encrypted:	 b'02c9b6ed4ffcf2bdf4f3118ecc232326'
Decrypted:	 Hello
